In [18]:
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
import numpy as np
import geopandas as gpd
from shapely.geometry import box
from pyproj import Proj, transform


In [11]:

data_path = "./Landsat8_Helheim/2021-06-14/"

# File paths to your R, G, and B band images
red_band_path = f"{data_path}LC08_L2SP_232013_20210614_20210622_02_T1_SR_B2.TIF"
green_band_path = f"{data_path}LC08_L2SP_232013_20210614_20210622_02_T1_SR_B3.TIF"
blue_band_path = f"{data_path}LC08_L2SP_232013_20210614_20210622_02_T1_SR_B4.TIF"

In [19]:
# Open the individual band files
with rasterio.open(red_band_path) as red:
    red_data = red.read(1)
    profile = red.profile  # Save profile to use later
    print(red.crs) # Show the CRS

with rasterio.open(green_band_path) as green:
    green_data = green.read(1)

with rasterio.open(blue_band_path) as blue:
    blue_data = blue.read(1)

# Stack the bands into an array
stacked_data = np.stack([red_data, green_data, blue_data])

# Update profile to reflect the new number of bands
profile.update(dtype=rasterio.uint8, count=3)

EPSG:32624


In [20]:
# Specify coordinates to crop (xmin, ymin, xmax, ymax)
# Helheim ~= 66.35°N and longitude -38.2°W
#crop_coords = (-38.3986,66.2599,-38.0014,66.4401) # 20Km box derived from GPT
crop_coords = (527017.08,7349009.52,544539.09,7369322.04) #converted to CRS EPSG:32624 (UTM Zone 24N)
geom = box(*crop_coords)

# Convert geometry to GeoDataFrame for rasterio masking
gdf = gpd.GeoDataFrame({"geometry": [geom]}, crs=red.crs)

In [22]:
FileOut = "Helheim-2021-06-14_rgb.tif"

# Crop the combined image
with rasterio.open(FileOut, 'w', **profile) as dst:
    dst.write(stacked_data)

# Read and crop the combined image
with rasterio.open(FileOut) as src:
    out_image, out_transform = mask(src, gdf.geometry, crop=True)
    out_meta = src.meta.copy()

# Update the metadata after cropping
out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

# Save the cropped image
with rasterio.open(FileOut, 'w', **out_meta) as dest:
    dest.write(out_image)